# 형태소 분석
- 텍스트 데이터에서 단어를 이루는 가장 작은 의미 단위인 형태소(Morpheme)를 분석하고 추출하는 과정
- 자연어 처리(NLP)의 기초 단계로, 문장을 구성하는 단어를 분해하고, 각각의 단어가 가지는 의미적, 문법적 정보를 분석하는 데 사용
- 형태소를 비롯하여, 어근, 접두사/접미사, 품사 등 다양한 언어적 속성의 구조를 파악



## 형태소란?
- 의미를 가지는 가장 작은 언어 단위
- 예:
    - 한국어: "사람들" → "사람" (명사) + "들" (복수 접미사)
    - 영어: "unhappy" → "un" (부정 접두사) + "happy" (형용사)
- 형태소는 크게 두가지로 나뉜다.
    - 자립형태소: 혼자서도 의미를 가지는 형태소
    > 예: 명사, 동사, 형용사 등
    - 의존형태소: 단독으로는 의미를 가지지 못하고 다른 형태소와 결합하여 의미를 가지는 형태소
    > 예: 조사, 접사, 어미 등

## 언어별 특징
- 한국어
    - 조사가 붙는 교착어로, 한 단어에 여러 형태소가 결합
    > 예: "사람들이" → "사람" (명사) + "들" (복수) + "이" (주격 조사)
    - 조사, 어미 등이 문법적으로 중요한 역할을 하므로, 형태소 분석이 NLP에 필수
- 영어
    - 분석이 비교적 단순하며, 주로 접사(un-, -ed)나 동사 변화 분석에 집중
    > 예: "running" → "run" (어근) + "-ing" (현재분사 접미사)

## 형태소 분석기
- 형태소 분석기는 품사를 태깅해주는 라이브러리  
- 영어에서의 품사는 문장에서 단어들의 위치가 띄어쓰기 단위로 되어 있기 때문에 POS(Part of Speech) tagger라고 함
- 반면에 한국어에서는 단어를 다 잘라내야 제대로 형태소를 갈라낼 수 있어서 Morphology Analyzer(형태학적 분석)라고 함
  


# 뉴스 분류 데이터셋
- 학습 데이터
    - https://drive.google.com/file/d/1-DFxEF9otbqt-swnM1fXVWAFIweiR1qM/view?usp=sharing
- 테스트 데이터
    - https://drive.google.com/file/d/1rL-LkmmM46V2HmLI1CKxK8LbdpP2pzvT/view?usp=sharing

- 0 : 세계, 1: 스포츠. 3: 비즈니스, 4: 과학기술

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/멋쟁이사자차럼/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# nltk
- python에서 가장 오래되고 유명한 자연어 처리 라이브러리
- 영어 품사 정보
    - https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [3]:
import nltk
nltk.download('punkt_tab') # 토크나이저 모델
nltk.download('stopwords') # 불용어 리스트
nltk.download('averaged_perceptron_tagger_eng') # 품사정보

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [4]:
train = pd.read_csv(f"{DATA_PATH}train_news.csv")
test = pd.read_csv(f"{DATA_PATH}test_news.csv")

train.shape, test.shape

((89320, 3), (38280, 2))

In [5]:
train.head()

,title,desc,target
0,Sudan Postpones Decision to Expel Oxfam and Sa...,Sudan has decided to postpone a decision to ex...,0
1,Coming Soon: Mobile TV,Cell phone manufacturers are teaming up to bri...,2
2,Experts warn of Internet flu vaccine scam,Although the United States is experiencing a s...,3
3,Bollor ups Havas stake to 20.2,Corporate raider Vincent Bollor said yesterday...,2
4,"Hurricane Ivan Kills 20 in Grenada, Heads West...",Reuters - Hurricane Ivan killed at least 20 pe...,0


In [6]:
test.head()

,title,desc
0,Mass. launches insurance probes,Massachusetts Attorney General Thomas F. Reill...
1,Jackson the Wizard of Loz,WHATEVER her status as an individual in the wo...
2,Coffee-Based Log Burns Cleaner -- But No Starb...,"Take an entrepreneur, add an interesting fact ..."
3,Annual Cell Phone Guide,Fast Forward columnist Rob Pegoraro was online...
4,Casino workers end strike in Atlantic City,"ATLANTIC CITY, NJ -- Thousands of cocktail wai..."


In [7]:
text = train["desc"].loc[0]
text

'Sudan has decided to postpone a decision to expel the heads of two British aid agencies - Oxfam and Save the Children - citing administrative difficulties and humanitarian grounds.'

## 토큰화

- 문장이나 텍스트 데이터를 작은단위로 나누는 과정
- 텍스트를 컴퓨터가 바로 이해할 수 없기 때문에 딥러닝 모델에 입력하기 위해 숫자데이터로 변환해야한다. 토큰화가 이 첫걸음

In [8]:
from nltk.tokenize import word_tokenize
word_tokenize(text)

['Sudan',
 'has',
 'decided',
 'to',
 'postpone',
 'a',
 'decision',
 'to',
 'expel',
 'the',
 'heads',
 'of',
 'two',
 'British',
 'aid',
 'agencies',
 '-',
 'Oxfam',
 'and',
 'Save',
 'the',
 'Children',
 '-',
 'citing',
 'administrative',
 'difficulties',
 'and',
 'humanitarian',
 'grounds',
 '.']

## 불용어

In [9]:
from nltk.corpus import stopwords # stopwords 불용어 사전
stopwords.words("english")[:10]  # 영어 불용어 리스트 10개 출력

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [10]:
len( stopwords.words("english") )

179

## 품사 태깅

In [11]:
tokens = word_tokenize(text)  # 일단 text 토큰화하기
nltk.pos_tag(tokens)

[('Sudan', 'NNP'),
 ('has', 'VBZ'),
 ('decided', 'VBN'),
 ('to', 'TO'),
 ('postpone', 'VB'),
 ('a', 'DT'),
 ('decision', 'NN'),
 ('to', 'TO'),
 ('expel', 'VB'),
 ('the', 'DT'),
 ('heads', 'NNS'),
 ('of', 'IN'),
 ('two', 'CD'),
 ('British', 'JJ'),
 ('aid', 'NN'),
 ('agencies', 'NNS'),
 ('-', ':'),
 ('Oxfam', 'NNP'),
 ('and', 'CC'),
 ('Save', 'NNP'),
 ('the', 'DT'),
 ('Children', 'NNP'),
 ('-', ':'),
 ('citing', 'VBG'),
 ('administrative', 'JJ'),
 ('difficulties', 'NNS'),
 ('and', 'CC'),
 ('humanitarian', 'JJ'),
 ('grounds', 'NNS'),
 ('.', '.')]

- N or V or J 로 시작하는 품사들의 단어들만 다시 새로운 리스트로 담고 싶다면?

In [12]:
result = [ t for t, pos in nltk.tag.pos_tag(tokens) if pos[0] in ["N", "V", "J"] ]
result

['Sudan',
 'has',
 'decided',
 'postpone',
 'decision',
 'expel',
 'heads',
 'British',
 'aid',
 'agencies',
 'Oxfam',
 'Save',
 'Children',
 'citing',
 'administrative',
 'difficulties',
 'humanitarian',
 'grounds']

In [13]:
train["clean"] = train["desc"].str.replace("[^\w ]+", "", regex=True).str.lower()
test["clean"] = test["desc"].str.replace("[^\w ]+", "", regex=True).str.lower()

## 표제어

- 기본형, 사전형 뜻함

In [14]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [15]:
from nltk.stem import WordNetLemmatizer

In [16]:
wnl = WordNetLemmatizer()
wnl.lemmatize("dogs")

'dog'

In [17]:
wnl.lemmatize("is")

'is'

In [18]:
wnl.lemmatize("has")

'ha'

- "v"(동사) , "a"(형용사), "n"(명사)

In [19]:
wnl.lemmatize("has", "v")

'have'

# spacy
- 딥러닝 기반의 형태소 분석 라이브러리

In [20]:
import spacy

In [21]:
nlp = spacy.load("en_core_web_sm") # 영어 모델 객체 로딩
nlp

In [22]:
text = train["clean"].iloc[0]
text

'sudan has decided to postpone a decision to expel the heads of two british aid agencies  oxfam and save the children  citing administrative difficulties and humanitarian grounds'

- nlp(text) : 텍스트를 처리하여 Doc 객체를 생성

In [23]:
doc = nlp(text)
type(doc) # Doc 객체

spacy.tokens.doc.Doc

In [24]:
len(doc)  # 29개의 토큰 객체

29

In [25]:
doc[1] # 인덱싱 가능!!

has

In [26]:
type(doc[1]) # 토큰 객체

spacy.tokens.token.Token

In [27]:
doc[1].text # 원래 단어

'has'

In [28]:
doc[1].lemma_ # 표제어

'have'

In [29]:
doc[1].tag_ # 품사

'VBZ'

In [30]:
doc[1].is_alpha # 알파벳 여부

True

In [31]:
doc[1].is_stop # 불용어 여부

True

In [32]:
cols = ["단어","표제어","품사","알파벳여부","불용어여부"]
data = [ [token.text, token.lemma_, token.tag_, token.is_alpha, token.is_stop]   for token in doc ]
pd.DataFrame(data, columns=cols)

,단어,표제어,품사,알파벳여부,불용어여부
0,sudan,sudan,NNP,True,False
1,has,have,VBZ,True,True
2,decided,decide,VBN,True,False
3,to,to,TO,True,True
4,postpone,postpone,VB,True,False
5,a,a,DT,True,True
6,decision,decision,NN,True,False
7,to,to,TO,True,True
8,expel,expel,VB,True,False
9,the,the,DT,True,True


- tokenizer 메서드 사용시 속도는 빠르나, 품사나 표제어 같은 정보를 추출할 수 없음
- tokenizer 는 단순히 텍스트를 토큰화만 수행

In [33]:
doc = nlp.tokenizer(text)
doc

sudan has decided to postpone a decision to expel the heads of two british aid agencies  oxfam and save the children  citing administrative difficulties and humanitarian grounds

In [34]:
doc[1]

has

In [35]:
doc[1].lemma_

''

In [36]:
doc[1].tag_

''

- 품사가 N, V, J, R 시작하는 토큰들만 토큰화 하기

In [37]:
# train_list = []

# for text in tqdm(train["clean"]):
#     doc = nlp(text)
#     tmp = [ t for t in doc if t.tag_[0] in "NVJR" ]
#     train_list.append(tmp)

In [38]:
train_list = []
for text in tqdm(train["clean"]):
    doc = nlp.tokenizer(text)
    tmp = [ t for t in doc if not t.is_alpha ]
    train_list.append(tmp)

  0%|          | 0/89320 [00:00<?, ?it/s]

- nltk 활용해서 모든 문서에 대해 불용어 제거와 명사, 동사, 형용사, 부사만 토큰화해서 train_list 에 담아주세요.
    - test_list에도 동일하게 작업해 주세요.

In [39]:
train_list = []
stop_words = stopwords.words("english")

for text in tqdm(train["clean"]):
    tokens = word_tokenize(text)
    tokens = nltk.pos_tag(tokens)
    tokens = [ t for t, p in tokens if t not in stop_words and  p[0] in "NVJR"  ]
    train_list.append(  " ".join(tokens)  )

  0%|          | 0/89320 [00:00<?, ?it/s]

In [40]:
test_list = []
for text in tqdm(test["clean"]):
    tokens = word_tokenize(text)
    tokens = nltk.pos_tag(tokens)
    tokens = [ t for t, p in tokens if t not in stop_words and  p[0] in "NVJR"  ]
    test_list.append( " ".join(tokens) )

  0%|          | 0/38280 [00:00<?, ?it/s]


> ***CountVectorizer***



1. **어휘집(Vocabulary):**  
```plaintext
Vocabulary: ['and' 'language' 'learning' 'love' 'natural']
```

2. **문서-단어 행렬:**  
```plaintext
Document-Term Matrix:
 [[0 1 0 1 1]   # 첫 번째 문장
  [0 1 0 0 1]   # 두 번째 문장
  [1 0 1 1 0]]  # 세 번째 문장
```

- **행**: 각 문서 (3개의 문서)  
- **열**: 어휘집에 있는 단어 (5개의 단어)  
- **값**: 각 단어가 해당 문서에 등장한 횟수  

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorize - 텍스트 데이터를 숫자(벡터)로 변환하는 도구 (BoW 방식으로 벡터화)
# 출현 빈도를 기반으로 벡터화, 가장 많이 등장하는 500개만 어휘집에 포함시킴.
# 문서-단어 행렬 생성, 행: 문서/ 열: 어휘집의 단어/ 값 : 단어 출현 빈도
# 단어 토큰화 → 어휘집 생성 → 단어 빈도 계산 → 문서-단어 행렬 생성
vec = CountVectorizer(max_features=500)
train_data = vec.fit_transform(train_list).A

In [42]:
train_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [43]:
test_data = vec.transform(test_list).A # test는 절대 fit 하면 안됨
test_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

- 스케일링

In [44]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

- 정답 데이터

In [45]:
train['target'].nunique() # 다중분류! 분류해야할게 4개다

4

In [46]:
train['target'].to_numpy()

array([0, 2, 3, ..., 2, 0, 0])

In [47]:
target = train['target'].to_numpy()

In [48]:
target.shape, target.dtype

((89320,), dtype('int64'))

# 데이터셋

- 전이학습에서 복사. transform 은 여기서 안했으니까 필요 없음.

In [49]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self,  x, y=None):
      self.x , self.y = x, y

    def __len__(self):
      return self.x.shape[0]

    def __getitem__(self, i):
      item = {}
      # 여기서 왜 tensor가 아니라 Tensor ?
      # torch.Tensor() 기본적으로 float32로 데이터를 변환
      # PyTorch 신경망 레이어 입력데이터의 dtype를 float32로 요구
      item["x"] = torch.Tensor(self.x[i])
      if self.y is not None:
           item["y"] = torch.tensor(self.y[i])
      return item

In [50]:
dt = NewsDataset(train_data,target)
dt[0]   # 여기선 대괄호가 1개 있어야한다.. 왜? 이미지가 아니라서 그런가?

{'x': tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3333, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000,
         0.0000, 0.0000

In [51]:
dl= torch.utils.data.DataLoader(dt, batch_size=2)

In [52]:
batch = next(iter(dl))
batch

{'x': tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3333, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000,
          0

# 모델 클래스

In [53]:
train_data.shape, test_data.shape, target.shape

((89320, 500), (38280, 500), (89320,))

In [54]:
# class Net(torch.nn.Module):
#     def __init__(self,in_features):
#         super().__init__()
#         self.seq = torch.nn.Sequential(
#             torch.nn.Linear(in_features, in_features//2),
#             torch.nn.ReLU(),
#             torch.nn.Linear(in_features//2, in_features//4),
#             torch.nn.ReLU(),
#             torch.nn.Linear(in_features//4,4)
#         )

#     def forward(self, x):
#         return self.seq(x)

In [55]:
# Net(train_data.shape[1])(batch['x'])

In [57]:
# # 우리만의 커스텀 레이러를 만들자꾸나!
# # 잔차 연결 활용하는 구조
# class ResidualBlock(torch.nn.Module):
#   def __init__(self, in_features):
#     super().__init__()
#     self.fx = torch.nn.Sequential(
#             torch.nn.Linear(in_features, in_features//2),
#             torch.nn.ReLU(),  # 음수 값을 0으로 만들고 양수는 그대로 통과
#             torch.nn.Dropout(0.5), # 50% 뉴런 랜덤하게 비활성화
#             # 잔차 연결 구현할때 입력과 출력의 차원을 동일하게 유지해야 더하기 연산 가능
#             # in_features//2 에서 차원 복원
#             torch.nn.Linear(in_features, in_features)
#     )
#     self.relu = torch.nn.ReLU()

#   def forward(self,x):
#     fx = self.fx(x)
#     hx = fx+x  # 입력 x 와 네트워크 출력 fx 더함.
#     return self.relu(hx)

In [64]:
class ResidualBlock(torch.nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.fx = torch.nn.Sequential(
            torch.nn.Linear(in_features, in_features),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(in_features, in_features)
        )
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        fx = self.fx(x)
        hx = fx + x
        return self.relu(hx)

In [76]:
class Net(torch.nn.Module):
  def __init__(self, in_features):
    super().__init__()
    # 깊은 네트워크에서 발생하는 기울기 소실 문제 해결, 학습 더 안정적으로 만듬
    self.fx = torch.nn.Sequential(
        ResidualBlock(in_features), # 특성 변환 및 잔차연결
        ResidualBlock(in_features), # 특성 변환 및 잔차연결
        ResidualBlock(in_features), # 특성 변환 및 잔차연결
        ResidualBlock(in_features)  # 특성 변환 및 잔차연결
    )
    self.output_layer = torch.nn.Linear(in_features, 4)  # 4개의 클래스 예측

  def forward(self,x):
    # x = self.seq(x) -> __init__메서드에서 정의되지 않은 변수 사용 : 에러
    x = self.fx(x)
    return self.output_layer(x)

In [65]:
# class Net(torch.nn.Module):
#     def __init__(self, in_features, n_layers=8):
#         super().__init__()

#         self.init_layer = torch.nn.Sequential(
#             torch.nn.Linear(in_features, in_features // 2),
#             torch.nn.BatchNorm1d(in_features // 2),
#             torch.nn.LeakyReLU()
#         )
#         res_list = [ ResidualBlock(in_features//2) for _ in range(n_layers) ]
#         self.seq = torch.nn.Sequential(*res_list)
#         self.output_layer = torch.nn.Linear(in_features//2, 4)

#     def forward(self, x):
#         x = self.init_layer(x)
#         x = self.seq(x)
#         return self.output_layer(x)

In [77]:
Net(train_data.shape[1])(batch["x"])

tensor([[ 0.0044, -0.0519, -0.0284, -0.0178],
        [ 0.0400, -0.0431, -0.0197,  0.0018]], grad_fn=<AddmmBackward0>)

# train loop

In [78]:
def train_loop(dl, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train() # 모델을 학습모드로 설정하는 메서드
    for batch in dl:
        pred = model(batch["x"].to(device))
        loss = loss_fn(pred, batch["y"].to(device)) # 값 비교 해서 손실값 계산

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dl)
    return epoch_loss

# test loop

In [79]:
@torch.no_grad()
def test_loop(dl, model, loss_fn, device):
    epoch_loss = 0  # 가중치 업데이트 할꺼 아니니까 ㅣㄹ요가 없다
    model.eval()

    act = torch.nn.Softmax()
    pred_list = []
    for batch in dl:
        pred = model( batch["x"].to(device) )
        if batch.get("y") is not None:
            loss = loss_fn(pred, batch["y"].to(device) )
            epoch_loss += loss.item()

        pred = act(pred) # 예측값에 활성화 함수(act) 적용하는 코드 - 비선형성 추가된 출력값 생성
        pred = pred.to("cpu").numpy() # cpu로 옮기기, numpy로 변환
        pred_list.append(pred) # 예측값 list 에 추가

    pred = np.concatenate(pred_list)
    epoch_loss /= len(dl)
    return epoch_loss, pred

# 학습하기

In [80]:
train['target'].value_counts(normalize=True)

,proportion
target,
0,0.25
2,0.25
3,0.25
1,0.25


In [81]:
n_splits = 5
batch_size = 32
epochs = 100
loss_fn = torch.nn.CrossEntropyLoss() # 다중분류

from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
cv = KFold(n_splits, shuffle=True, random_state=SEED)

**Net() 호출할때 in_features 인자 전달 안해줘서 에러 났었다.**
- 에러 났던 당시 코드
```
 model = Net().to(device)
 ```

In [82]:
is_holdout = True
reset_seeds(SEED)
score_list = []
for i, (tri, vai) in enumerate(cv.split(train_data)): # train_data 말고 train 넣어 줘도 상관 없다.
    # 학습 데이터
    train_dt = NewsDataset(train_data[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    # 검증 데이터
    valid_dt = NewsDataset(train_data[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    # 모델 객체 및 옵티마이저 생성
    model = Net(train_data.shape[1]).to(device)
    optimizer = torch.optim.Adam( model.parameters() )

    patience = 0 # 조기 종료 조건을 주기 위한 변수
    best_score = 0 # 현재 최고점수
    for _ in tqdm(range(epochs)):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)
        pred = np.argmax(pred, axis =1)
        score = f1_score(target[vai], pred, average='micro')
        patience += 1

        if score > best_score:
            best_score = score
            patience = 0
            torch.save( model.state_dict(), f"model{i}.pt" )

        if patience == 5:
            break

    score_list.append(best_score)
    print(f"f1-macro 최고점수: {best_score}")

    if is_holdout:
        break

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules

f1-macro 최고점수: 0.8333519928347515


# 테스트 데이터 예측

In [86]:
test_dt = NewsDataset(train_data)
test_dl = torch.utils.data.DataLoader(test_dt, shuffle=False, batch_size=batch_size)


pred_list = []
for i in range(n_splits):
    model = Net(train_data.shape[1]).to(device)
    state_dict = torch.load(f"model{0}.pt", weights_only=True)
    model.load_state_dict(state_dict)

    _, pred = test_loop(test_dl, model, None, device)
    pred_list.append(pred)

In [85]:
pred = np.mean(pred_list, axis=0)
pred =np.argmax(pred,axis=1)
pred.shape

(89320,)

# 기타

1. 피처를 천개 이천개까지 늘려서 컴볼루션 레이어 한테 해보라고 층을 쌓아 보기.
  - 입력데이터(특성, 피처)의 개수를 1,000개, 2,000개와 같이 많게 늘려도 CNN 레이어를 적용 가능
  - CNN 레이어는 이미지 데이터 뿐만 아니라 정형 데이터에도 적용 가능
2. 무조건 정형데이터라고 flat connected layer만 사용할것이냐? > 아니다
  - CNN, RNN, Transformer와 같은 다양한 레이어를 시도해볼 수 있다는 의미
  - Convolutional Layer도 사용해볼 수 있다.

## **1. CNN (Convolutional Neural Network) → "돋보기로 부분 부분 살펴보는 탐정"**  

### **비유**  
CNN은 **돋보기**를 사용해 이미지를 **부분적으로(국소적)** 살펴보는 탐정과 비슷합니다.  
- 전체를 한 번에 보는 게 아니라 **일부 영역(패치)**를 집중해서 관찰하면서 중요한 패턴을 찾아냅니다.  
- 여러 개의 돋보기(커널)가 다른 **패턴**을 감지합니다. 예를 들어 이미지에서 **수평선, 수직선, 원** 같은 것들이죠.  

### **예시**  
- **이미지 인식**: 탐정이 사람의 얼굴을 보면서 **눈, 코, 입** 같은 국소적인 특징들을 하나하나 감지합니다.  
- **시계열 데이터**: 특정 구간에서 반복되는 패턴이나 이상 징후를 찾아냅니다.

---

## **2. RNN (Recurrent Neural Network) → "줄줄이 이야기를 이어가는 스토리텔러"**  

### **비유**  
RNN은 이야기를 순서대로 들으면서 **앞에 나왔던 내용**을 기억해 가며 다음 이야기를 이해하는 **스토리텔러**입니다.  
- 이야기를 **하나씩 순서대로 처리**합니다.  
- 과거에 들었던 내용을 **메모리**에 저장하고, 새로운 이야기에 반영합니다.  

### **예시**  
- **텍스트 처리**: "나는 오늘 아침에 **빵**과 커피를 먹었어"라는 문장을 처리할 때, "빵"이 나온 후 "먹었어"라는 단어를 만났을 때 **맥락**을 이해합니다.  
- **시계열 데이터**: 주식 데이터를 시간 순서대로 처리하면서 **과거 가격**을 기억해 미래 가격을 예측합니다.

---

## **3. Transformer → "동시 다발적으로 전체를 살펴보는 회의 진행자"**  

### **비유**  
Transformer는 회의실에서 **모든 사람의 발언을 동시에 듣고** 어떤 이야기가 중요한지 파악하는 **회의 진행자**와 비슷합니다.  
- RNN은 순서대로 이야기를 듣지만, Transformer는 **동시에 모든 내용을 보고** 각 부분이 서로 얼마나 관련 있는지 평가합니다.  
- 중요한 부분에 **집중(attention)**합니다.  

### **예시**  
- **번역 작업**: Transformer는 "I love you"를 번역할 때 "love"가 "I"와 "you" 둘 다 관계있음을 동시에 파악합니다.  
- **텍스트 요약**: 긴 문서의 모든 문장을 동시에 보고 가장 중요한 내용을 찾아냅니다.

---

## **4. Fully Connected Layer → "모든 사람을 연결하는 대형 파티"**  

### **비유**  
Fully Connected Layer는 모든 사람이 서로 이야기를 주고받는 **대형 파티장**과 같습니다.  
- 모든 입력(특성)과 모든 출력이 **완전히 연결**되어 있습니다.  
- 하나의 정보를 나머지 모든 정보와 결합해 새로운 관계를 학습합니다.  

### **예시**  
- **정형 데이터**: 설문 조사 데이터에서 모든 질문(특성)이 각각 다른 답변(결과)과 연결됩니다.  
- **마지막 레이어**: 다른 네트워크(CNN, RNN)가 특징을 추출한 후, 결과를 도출할 때 Fully Connected Layer를 사용합니다.

---

## **종합 비교**

| **구분**             | **비유**                          | **특징**                                    | **사용 예**                       |
|----------------------|----------------------------------|-------------------------------------------|---------------------------------|
| **CNN**             | 돋보기로 부분적으로 탐색하는 탐정 | 국소적 패턴을 찾아내고 효율적으로 학습        | 이미지 인식, 시계열 패턴 감지      |
| **RNN**             | 스토리텔러                       | 순서대로 데이터를 처리하며 과거 정보를 기억     | 텍스트 처리, 시계열 예측           |
| **Transformer**     | 회의 진행자                      | 모든 데이터를 동시에 살피고 중요한 것에 집중   | 번역, 텍스트 요약, NLP 전반        |
| **Fully Connected**  | 모든 사람을 연결하는 대형 파티    | 모든 특성을 서로 연결해 관계를 학습           | 정형 데이터 분석, 출력 레이어       |

---

## **요약**

- **CNN**: 돋보기처럼 **부분적으로 패턴을 탐색**하며 중요한 특징을 학습합니다.  
- **RNN**: 스토리텔러처럼 **순서대로 데이터**를 처리하고 과거의 정보를 기억합니다.  
- **Transformer**: 회의 진행자처럼 **전체 데이터를 동시에** 보고 중요한 부분을 파악합니다.  
- **Fully Connected**: 대형 파티장처럼 **모든 입력과 출력을 연결**하여 관계를 학습합니다.

이제 감이 좀 잡히셨나요? 상황에 따라 각 네트워크를 조합하면 더 강력한 모델을 만들 수 있습니다. 😊